# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

**Описание проекта**
    Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
    В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.
    Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.
Инструкция по выполнению проекта

**Описание данных**
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
сalls — количество звонков,
minutes — суммарная длительность звонков в минутах,
messages — количество sms-сообщений,
mb_used — израсходованный интернет-трафик в Мб,
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
display(df)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [6]:
df.duplicated().sum()

0

**Вывод**
1. Данные без пропусков и повторений
2. Предобработка данных не требуется.

## Разбейте данные на выборки

In [7]:
train, test = train_test_split(df, train_size=0.6, test_size=0.4, random_state=12345)

In [8]:
features = test.drop('is_ultra', axis=1)
target = test['is_ultra']

In [9]:
features_val, features_test, target_val, target_test = train_test_split(features, target, train_size=0.5, test_size=0.5, random_state=12345)

Сначала делим датафрейм на две части в соотношении 60/40, а затем тестовую выборку в размере 40% от датафрейма делим пополам, получая окончательные валидационную и тестовую выборки.

In [10]:
features_train = train.drop('is_ultra', axis=1)
target_train = train['is_ultra']

In [11]:
print(features_train)
print(target_train)

      calls  minutes  messages   mb_used
3027   60.0   431.56      26.0  14751.26
434    33.0   265.17      59.0  17398.02
1226   52.0   341.83      68.0  15462.38
1054   42.0   226.18      21.0  13243.48
1842   30.0   198.42       0.0   8189.53
...     ...      ...       ...       ...
2817   12.0    86.62      22.0  36628.85
546    65.0   458.46       0.0  15214.25
382   144.0   906.18       0.0  25002.44
2177   38.0   301.27      37.0  28914.24
482   160.0  1084.05      95.0  36843.92

[1928 rows x 4 columns]
3027    0
434     0
1226    0
1054    0
1842    0
       ..
2817    1
546     1
382     1
2177    1
482     1
Name: is_ultra, Length: 1928, dtype: int64


**Создал 3 выборки**

features_train, target_train - *обучающая выборка*

features_val, target_val - *валидационная выборка*

features_test, target_test - *тестовая выборка*



## Исследуйте модели

**Дерево решений**

In [12]:
best_accuracy = 0
best_depth = 0
for depth in range(1,10):
    model = DecisionTreeClassifier(max_depth=depth, random_state=321)
    model.fit(features_train, target_train)
    accuracy = model.score(features_val, target_val)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth
print("Глубина дерева:", best_depth)
print("Качество:", best_accuracy)

Глубина дерева: 9
Качество: 0.7869362363919129


**Логистическая регрессия**

In [13]:
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy = model.score(features_val, target_val)
print("Качество:", accuracy)

Качество: 0.7589424572317263


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


**Случайный лес**

In [14]:
best_accuracy = 0
best_depth = 0
for depth in range(1,10):
    for est in range(1,20,2):
        for sample in range(1,10):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, min_samples_leaf=sample, random_state=321)
            model.fit(features_train, target_train)
            accuracy = model.score(features_val, target_val)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth) 
print("Количество деревьев:", best_est)      
print("Минимально объектов в узле:", best_sample)      
print("Качество:", best_accuracy)     

Глубина дерева: 6
Количество деревьев: 5
Минимально объектов в узле: 2
Качество: 0.8040435458786936


**Вывод**
1. Провел ислледование 3 различных моделей машинного обучения.
2. Наилучший результат показала 3-я модель "Случайный лес" с результатом 0.8040435458786936

## Проверьте модель на тестовой выборке

**Дерево решений**

In [15]:
model = DecisionTreeClassifier(max_depth=10, random_state=321)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7791601866251944


**Логистическая регрессия**

In [16]:
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7402799377916018


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


**Случайный лес**

In [17]:
#model = RandomForestClassifier(max_depth=9, n_estimators=40, min_samples_leaf=4, random_state=321)
#model.fit(features_train, target_train)
#accuracy = model.score(features_test, target_test)
#print("Качество:", accuracy)

In [18]:
#model = RandomForestClassifier(max_depth=9, n_estimators=42, min_samples_leaf=3, random_state=321)
#model.fit(features_train, target_train)
#accuracy = model.score(features_test, target_test)
#print("Качество:", accuracy)

In [19]:
#model = RandomForestClassifier(max_depth=8, n_estimators=41, min_samples_leaf=4, random_state=321)
#model.fit(features_train, target_train)
#accuracy = model.score(features_test, target_test)
#print("Качество:", accuracy)

In [20]:
#model = RandomForestClassifier(max_depth=9, n_estimators=39, min_samples_leaf=3, random_state=321)
#model.fit(features_train, target_train)
#accuracy = model.score(features_test, target_test)
#print("Качество:", accuracy)

In [21]:
model = RandomForestClassifier(max_depth=9, n_estimators=41, min_samples_leaf=3, random_state=321)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.8055987558320373


**Вывод**
1. Проверил модель на 3х тестовых выборках
2. Определил наилучшие гиперпараметры в RandomForest max_depth=9, n_estimators=41, min_samples_leaf=3
(для примера оставил только 3, по факту перебрал больше 30 параметров)
3. Наилучший результат 0.8055987558320373

## (бонус) Проверьте модели на адекватность

In [22]:
dmb = DummyClassifier(strategy="stratified" ,random_state=321)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Качество прогноза :", acc)
print("Качество этого прогноза :", accuracy)

Качество прогноза : 0.5816485225505443
Качество этого прогноза : 0.8055987558320373


In [23]:
dmb = DummyClassifier(strategy='most_frequent', random_state=321)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Качество прогноза:", acc)
print("Качество этого прогноза:", accuracy)

Качество прогноза: 0.6842923794712286
Качество этого прогноза: 0.8055987558320373


In [24]:
dmb = DummyClassifier(strategy='uniform', random_state=321)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Качество прогноза :", acc)
print("Качество этого прогноза :", accuracy)

Качество прогноза : 0.48833592534992226
Качество этого прогноза : 0.8055987558320373


In [25]:
dmb = DummyClassifier(strategy='prior', random_state=321)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Качество прогноза :", acc)
print("Качество этого прогноза :", accuracy)

Качество прогноза : 0.6842923794712286
Качество этого прогноза : 0.8055987558320373


**Вывод**
1. Проверил модели на адекватность через DummyClassifier 4мя различными стратегиями

Стратегия использования для создания прогнозов:

«Стратифицированный»: генерирует прогнозы с учетом распределения классов обучающей выборки.
«Most_frequent»: всегда предсказывает наиболее частую метку в обучающем наборе.
«Prior»: всегда предсказывает класс, который максимизирует предыдущий класс (например, «most_frequent»), а predict_proba возвращает предыдущий класс.
«Uniform»: генерирует прогнозы равномерно в случайном порядке.э

2. Модель адекватна

# Общий вывод

Создал 3 выборки

features_train, target_train - обучающая выборка

features_val, target_val - валидационная выборка

features_test, target_test - тестовая выборка

Провел ислледование 3 различных моделей машинного обучения. Наилучший результат показала 3-я модель "Случайный лес" с результатом 0.8040435458786936.

Проверил модель на 3х тестовых выборках. Определил наилучшие гиперпараметры в RandomForest max_depth=9, n_estimators=41, min_samples_leaf=3 (для примера оставил только 1 наилучшую, по факту перебрал больше 30 параметров). Наилучший результат 0.8055987558320373

Проверил модели на адекватность через DummyClassifier 4мя различными стратегиями
Стратегия использования для создания прогнозов:

«Стратифицированный»: генерирует прогнозы с учетом распределения классов обучающей выборки. «Most_frequent»: всегда предсказывает наиболее частую метку в обучающем наборе. «Prior»: всегда предсказывает класс, который максимизирует предыдущий класс (например, «most_frequent»), а predict_proba возвращает предыдущий класс. «Uniform»: генерирует прогнозы равномерно в случайном порядке.

Модель адекватна